# Setup Geral

Se estiver executando este exercício no Google Colab, execute as próximas duas células. 

Caso esteja executando localmente, não é necessário executar mas certifique-se de que o **pyspark** está instalado e configurado em sua máquina.

In [ ]:
%%bash

# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Teste

O teste a ser realizado é composto de 3 partes:

- um exercício de programação em Python
- alguns exercícios de programação em SQL
- alguns exercícios de programação com PySpark

Você pode escolher qual das partes do exercício vai fazer primeiro. Todo o exercício deve ser completado no período de 48 horas.

# Python

In [7]:
# SETUP
nomes_alunos = [
    ('Maria', 1),
    ('João', 2),
    ('Pedro', 3),
    ('Gabriella', 4),
    ('Giovana', 5),
    ('Arthur', 6)
]

notas_alunos = {
    1: 9.5,
    2: 5.1,
    3: 8.7,
    4: 7.1,
    5: 4.8,
    6: 6.3
}

Implemente uma função que recebe uma lista de nomes de alunos, um dicionário de notas dos mesmo, sendo que essas estruturas se relacionam por um ID.

A função deve retornar em ordem alfabética, o nome dos alunos que obtiveram notas maior ou igual de uma nota de corte informada.

In [8]:
def filtra_alunos_acima_corte(alunos, notas, nota_corte):

    import logging

    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    
    try:
        if not __valida_variaveis_alunos_acima_corte(alunos, notas, nota_corte):
            return []

    except NameError:
        logging.warning("Adicionei uma checagem das variáveis ao teste, se for válido no teste, por favor executar a célula abaixo para sua criação")

                        
    dict_notas_aprovadas = dict(filter(lambda aluno: aluno[1] >= nota_corte, notas_alunos.items()))

    if not dict_notas_aprovadas:
        logging.warning("Nao temos alunos aprovados.")
        return []

    
    id_notas_aprovadas = list(dict_notas_aprovadas.keys())
    alunos_aprovados = list(filter(lambda x: x[1] in id_notas_aprovadas, alunos))

    if len(alunos_aprovados) < len(id_notas_aprovadas):
        logging.warning("Algum id de aluno aprovado não foi encontrado.")
    
    nome_alunos_aprovados = list(map(lambda x: x[0], alunos_aprovados))
    
    nome_alunos_aprovados.sort()
    
    return nome_alunos_aprovados

    
filtra_alunos_acima_corte(nomes_alunos, notas_alunos, 6)

['Arthur', 'Gabriella', 'Maria', 'Pedro']

In [3]:
def __valida_variaveis_alunos_acima_corte(alunos, notas, nota_corte):

    import logging

    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    
    
    if not isinstance(alunos, (list)):
        logging.error("Não recebemos uma lista de alunos.")
        return False


    elif not all(map(lambda x: 
                     isinstance(x, tuple) and 
                     len(x) == 2 and 
                     isinstance(x[0], str) and
                     isinstance(x[1], int), alunos)
                ):
        logging.error("O formato dos alunos enviados não está no padrão esperado. O padrão correto é ('nome_aluno',10)")
        return False


    if not isinstance(notas, (dict)):
        logging.error("Não recebemos um dicionário de notas.")
        return False


    if not all(map(lambda x: isinstance(x, (int)), notas.keys())) \
    or not all(map(lambda x: isinstance(x, (int, float)), notas.values())):
        logging.error("As notas enviadas não estão no padrão esperado. O padrão correto é {1:10} sendo o id do aluno e sua nota, respectivamente.")
        return False

    
    if not isinstance(nota_corte, (int, float)):
        logging.error("Nota de corte inválida. É esperado um valor numérico (inteiro ou decimal).")
        return False
    
    return True


# SQL

**Setup**


In [5]:
%%bash
#mkdir bases_teste

curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   1406      0 --:--:-- --:--:-- --:--:--  1405
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0   373k      0  0:00:01  0:00:01 --:--:--  373k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0   484k      0 --:--:-- --:--:-- --:--:--  484k


In [81]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [83]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

**Função para execução de queries**

In [38]:
def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

Para executar alguma consulta, basta colocar seu código sql dentro da função q como no exemplo abaixo:

```python
q("""
    SELECT *
    FROM usuarios
""")
```

e o resultado será exibido na tela.

---

Nesta parte da atividade, você vai trabalhar com três tabelas:

- produtos
- usuarios
- vendas

Use-as para responder às perguntas a seguir.

1) Qual foi a quantidade de vendas nos estados de Minas Gerais e São Paulo para cada ano e mês?

In [84]:
q("""
SELECT
 u.estado
,YEAR(data_compra) AS ano_compra
,MONTH(data_compra) AS mes_compra
,COUNT(*) AS qtd_venda
FROM
vendas AS v
INNER JOIN usuarios AS u ON u.cod_usuario = v.cod_usuario
WHERE
u.estado IN ("Minas Gerais", "São Paulo")
GROUP BY
 u.estado
,YEAR(data_compra)
,MONTH(data_compra)
ORDER BY
 u.estado
,YEAR(data_compra)
,MONTH(data_compra)
""",n = 100)

+------------+----------+----------+---------+
|estado      |ano_compra|mes_compra|qtd_venda|
+------------+----------+----------+---------+
|Minas Gerais|2018      |6         |1        |
|Minas Gerais|2018      |8         |1        |
|Minas Gerais|2018      |10        |1        |
|Minas Gerais|2018      |11        |1        |
|Minas Gerais|2018      |12        |2        |
|Minas Gerais|2019      |1         |2        |
|Minas Gerais|2019      |2         |1        |
|Minas Gerais|2019      |4         |2        |
|Minas Gerais|2019      |5         |1        |
|Minas Gerais|2019      |6         |1        |
|Minas Gerais|2019      |7         |2        |
|Minas Gerais|2019      |8         |1        |
|Minas Gerais|2019      |9         |2        |
|Minas Gerais|2019      |10        |1        |
|Minas Gerais|2019      |11        |1        |
|Minas Gerais|2019      |12        |3        |
|Minas Gerais|2020      |1         |1        |
|Minas Gerais|2020      |2         |1        |
|Minas Gerais

2) Quais são os usuários por Estado que mais compraram em todo o período analisado e qual foi o número de compras realizadas, a quantidade total de itens comprados e valor total pago por usuário?

In [59]:
q("""
SELECT 
 cod_usuario
,estado
,qtd_total_compras
,qtd_total_itens
,valor_total_pago
FROM (
    SELECT
     u.cod_usuario
    ,u.estado
    ,COUNT(*) AS qtd_total_compras
    ,ROUND(SUM(v.quantidade),2) AS qtd_total_itens
    ,ROUND(SUM(v.valor),2) AS valor_total_pago
    ,RANK() OVER(PARTITION BY u.estado ORDER BY COUNT(*) desc) as ranking_por_compra_cliente
    FROM
    vendas AS v
    INNER JOIN usuarios AS u ON u.cod_usuario = v.cod_usuario
    GROUP BY
     u.cod_usuario
    ,u.estado
) WHERE ranking_por_compra_cliente = 1
ORDER BY
 estado
,valor_total_pago DESC

""")

+-----------+-------------------+-----------------+---------------+----------------+
|cod_usuario|estado             |qtd_total_compras|qtd_total_itens|valor_total_pago|
+-----------+-------------------+-----------------+---------------+----------------+
|2186       |Amapá              |7                |61             |51359.34        |
|2981       |Bahia              |9                |70             |126282.4        |
|1787       |Ceará              |8                |79             |121494.67       |
|700        |Goiás              |9                |74             |142389.09       |
|2811       |Maranhão           |13               |117            |184101.89       |
|2340       |Mato Grosso        |9                |112            |165197.3        |
|679        |Mato Grosso do Sul |6                |102            |152271.96       |
|3090       |Minas Gerais       |9                |93             |95697.3         |
|2843       |Paraná             |10               |114           

3) Quais são os usuários que não fizeram nenhuma compra?

In [68]:
q("""
SELECT
 u.*
FROM
usuarios AS u  
LEFT JOIN vendas AS v ON u.cod_usuario = v.cod_usuario
WHERE 
u.cod_usuario IS NOT NULL /*Removendo linhas vazias que estão na tabela*/
AND v.cod_usuario IS NULL
ORDER BY
 u.cod_usuario
""")

+-----------+-------------------+------------------+-----------------------+-------------------+
|cod_usuario|data_cadastro      |faixa_etaria      |cidade                 |estado             |
+-----------+-------------------+------------------+-----------------------+-------------------+
|3499       |2019-12-17 00:00:00|Entre 10 a 15 anos|Jacuí                  |Minas Gerais       |
|3504       |2021-06-23 00:00:00|Entre 16 a 21 anos|Bom Jesus do Galho     |Minas Gerais       |
|3516       |2019-10-13 00:00:00|Entre 10 a 15 anos|Rio Verde              |Goiás              |
|3556       |2020-08-04 00:00:00|Entre 62 a 70 anos|Confresa               |Mato Grosso        |
|3560       |2018-09-04 00:00:00|Entre 16 a 21 anos|Maracaí                |São Paulo          |
|3564       |2018-07-02 00:00:00|Entre 62 a 70 anos|Betim                  |Minas Gerais       |
|3692       |2018-08-28 00:00:00|Mais de 70 anos   |Nipoã                  |São Paulo          |
|3754       |2021-05-16 00:00:

4) Qual é o ticket médio (média de valor gasto) e o número total de usuários que fizeram pelo menos uma compra por faixa etária?

In [78]:
q("""
SELECT
 u.faixa_etaria
,ROUND(AVG(v.valor),2) AS ticket_medio
,COUNT(DISTINCT u.cod_usuario) AS qtd_usuario
FROM
vendas AS v  
INNER JOIN usuarios AS u ON u.cod_usuario = v.cod_usuario
GROUP BY
 u.faixa_etaria
ORDER BY
 u.faixa_etaria
""")

+------------------+------------+-----------+
|faixa_etaria      |ticket_medio|qtd_usuario|
+------------------+------------+-----------+
|Entre 10 a 15 anos|8890.13     |7          |
|Entre 16 a 21 anos|12725.95    |11         |
|Entre 22 a 27 anos|12203.02    |10         |
|Entre 28 a 36 anos|8233.04     |9          |
|Entre 37 a 49 anos|16727.19    |6          |
|Entre 50 a 61 anos|10849.48    |12         |
|Entre 62 a 70 anos|10012.63    |7          |
|Mais de 70 anos   |11303.1     |4          |
+------------------+------------+-----------+



In [332]:
q("""
SELECT
 p.cod_produto
,p.categoria_produto
,p.nome_produto
,v.cod_usuario
,v.data_compra
,v.quantidade
,v.valor
FROM
vendas AS v
INNER JOIN produtos AS p 
    ON p.cod_produto = v.cod_produto
where
cod_usuario = 37
ORDER BY
 categoria_produto
,data_compra
,nome_produto
""")

+-----------+-------------------+--------------------------------+-----------+-------------------+----------+--------+
|cod_produto|categoria_produto  |nome_produto                    |cod_usuario|data_compra        |quantidade|valor   |
+-----------+-------------------+--------------------------------+-----------+-------------------+----------+--------+
|13         |Alimentos          |Kit 3 barras de chocolate Alpino|123        |2020-12-20 00:00:00|18        |178.2   |
|16         |Alimentos          |Biscoito Chocolicia Chocolate   |123        |2021-02-06 00:00:00|19        |95.0    |
|13         |Alimentos          |Kit 3 barras de chocolate Alpino|123        |2021-05-18 00:00:00|13        |128.7   |
|16         |Alimentos          |Biscoito Chocolicia Chocolate   |123        |2021-07-20 00:00:00|2         |10.0    |
|7          |Casa e bem-estar   |Ar-condicionado 9500 BTUs LG    |123        |2020-12-13 00:00:00|8         |10403.92|
|12         |Casa e bem-estar   |Umidificador de

In [85]:
spark.stop()

# PySpark

**setup**:

In [86]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   3539      0 --:--:-- --:--:-- --:--:--  3546
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0   553k      0 --:--:-- --:--:-- --:--:--  552k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0   372k      0 --:--:-- --:--:-- --:--:--  371k


In [87]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [267]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

Responda às perguntas a seguir utilizando **Spark DATAFRAMES**.

1) Qual foi o total de compras realizadas, o total de itens comprados e a receita total obtida em todo o período analisado?

In [171]:
from pyspark.sql.functions import format_string

df_vendas = spark.read.table("vendas")

df_resposta = df_vendas.agg(
    count("*").alias("total_compras"),
    sum("quantidade").alias("total_itens"),
    sum("valor").alias("receita_total"),
)

df_resposta = df_resposta.withColumn(
    "receita_total", format_string("%.0f", df_resposta["receita_total"])
)


df_resposta.show()


+-------------+-----------+-------------+
|total_compras|total_itens|receita_total|
+-------------+-----------+-------------+
|        20000|     209149|    215849002|
+-------------+-----------+-------------+



2) Quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga?

In [269]:
from pyspark.sql.functions import col, count, dense_rank, desc, format_number, sum
from pyspark.sql.window import Window

df_usuarios = spark.read.table("usuarios")
df_vendas = spark.read.table("vendas")
df_produtos = spark.read.table("produtos")

estados_considerados = [
    ("São Paulo",),
    ("Rio de Janeiro",),
    ("Minas Gerais",),
    ("Espírito Santo",),
    ("Paraná",),
    ("Santa Catarina",),
    ("Rio Grande do Sul",),
]


df_produtos = df_produtos["cod_produto", "nome_produto"]

df_estados_considerados = spark.createDataFrame(estados_considerados, ["estado"])


df_usuarios_nos_estados = df_usuarios.join(
    df_estados_considerados,
    df_usuarios.estado == df_estados_considerados.estado,
    "inner",
)


df_compras_por_usuario = df_usuarios_nos_estados.join(
    df_vendas, df_vendas.cod_usuario == df_usuarios_nos_estados.cod_usuario, "inner"
)


df_compras_por_usuario = df_compras_por_usuario[
    "usuarios.cod_usuario",
    "usuarios.estado",
    "cod_produto",
    "quantidade",
    "valor",
]


df_agrupado_por_produto_estado = df_compras_por_usuario.groupBy(
    "cod_produto", "estado"
).agg(sum("quantidade").alias("qtd_itens"), sum("valor").alias("valor_total"))


window_spec = Window.partitionBy("estado").orderBy(desc("qtd_itens"))
df_rank_por_produto_estado = df_agrupado_por_produto_estado.select(
    "cod_produto",
    "estado",
    "qtd_itens",
    "valor_total",
    dense_rank().over(window_spec).alias("rank"),
)


df_rank_por_produto_estado = df_rank_por_produto_estado.filter(col("rank") <= 3)

df_rank_por_produto_estado = df_rank_por_produto_estado.withColumn(
    "media_preco", col("valor_total") / col("qtd_itens")
)


df_resposta = df_rank_por_produto_estado.join(
    df_produtos,
    df_produtos.cod_produto == df_rank_por_produto_estado.cod_produto,
    "inner",
)

df_resposta = df_resposta[
    "vendas.cod_produto",
    "nome_produto",
    "estado",
    "qtd_itens",
    "valor_total",
    "rank",
    "media_preco",
]

df_resposta = df_resposta.withColumn("valor_total", format_number("valor_total", 2))
df_resposta = df_resposta.withColumn("media_preco", format_number("media_preco", 2))

df_resposta.show(truncate=False, n=100)


+-----------+--------------------------------+-----------------+---------+-----------+----+-----------+
|cod_produto|nome_produto                    |estado           |qtd_itens|valor_total|rank|media_preco|
+-----------+--------------------------------+-----------------+---------+-----------+----+-----------+
|15         |Alcool em Gel 70% Johnson       |Minas Gerais     |165      |823.35     |1   |4.99       |
|5          |Escrivaninha em L               |Minas Gerais     |78       |34,086.00  |2   |437.00     |
|1          |Notebook Asus Intel Core i7     |Minas Gerais     |71       |325,953.90 |3   |4,590.90   |
|11         |Jogo Mortal Kombat 11 PS4       |Paraná           |35       |1,746.50   |1   |49.90      |
|10         |Cadeira Gamer Xpress            |Paraná           |27       |18,897.30  |2   |699.90     |
|19         |SmartTV Samsung 4K 55 polegadas |Paraná           |25       |62,497.50  |3   |2,499.90   |
|15         |Alcool em Gel 70% Johnson       |Rio Grande do Sul|

3) Para cada produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago por eles? 

In [270]:
from pyspark.sql.functions import col, countDistinct, desc, format_number, min, max

df_vendas = spark.read.table("vendas")
df_produtos = spark.read.table("produtos")


df_vendas = df_vendas["cod_produto", "cod_usuario", "quantidade", "valor"]
df_vendas = df_vendas.withColumn("valor_unitario", col("valor") / col("quantidade"))

df_clientes_por_produto = df_vendas.groupBy("cod_produto").agg(
    min("valor_unitario").alias("valor_unitario_minimo"),
    max("valor_unitario").alias("valor_unitario_maximo"),
    countDistinct("cod_usuario").alias("qtd_usuarios"),
)

df_resposta = df_produtos.join(
    df_clientes_por_produto,
    df_produtos.cod_produto == df_clientes_por_produto.cod_produto,
    "left",
)

df_resposta = df_resposta.na.fill(
    0, subset=["qtd_usuarios", "valor_unitario_minimo", "valor_unitario_maximo"]
)

df_resposta = df_resposta.withColumn(
    "valor_unitario_minimo", format_number("valor_unitario_minimo", 2)
)
df_resposta = df_resposta.withColumn(
    "valor_unitario_maximo", format_number("valor_unitario_maximo", 2)
)

df_resposta = df_resposta[
    "produtos.cod_produto",
    "nome_produto",
    "qtd_usuarios",
    "valor_unitario_minimo",
    "valor_unitario_maximo",
]


df_resposta = df_resposta.orderBy(desc("qtd_usuarios"))
df_resposta.show(truncate=False, n=100)


+-----------+-----------------------------------+------------+---------------------+---------------------+
|cod_produto|nome_produto                       |qtd_usuarios|valor_unitario_minimo|valor_unitario_maximo|
+-----------+-----------------------------------+------------+---------------------+---------------------+
|18         |Lenço umedecido Turma da Monica    |880         |7.90                 |7.90                 |
|7          |Ar-condicionado 9500 BTUs LG       |863         |1,300.49             |1,300.49             |
|5          |Escrivaninha em L                  |860         |437.00               |437.00               |
|19         |SmartTV Samsung 4K 55 polegadas    |856         |2,499.90             |2,499.90             |
|9          |Microoondas 35L                    |847         |389.10               |389.10               |
|11         |Jogo Mortal Kombat 11 PS4          |847         |49.90                |49.90                |
|6          |Sofa 3 lugares          

4) Aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado.

In [357]:
from pyspark.sql.functions import asc, col, date_format, dense_rank, desc, format_number, when

from pyspark.sql.window import Window

df_vendas = spark.read.table("vendas")
df_produtos = spark.read.table("produtos")

df_vendas_por_categoria = df_vendas.join(
    df_produtos, df_vendas.cod_produto == df_produtos.cod_produto, "inner"
)

df_vendas_por_categoria = df_vendas_por_categoria[
    "cod_usuario",
    "produtos.cod_produto",
    "nome_produto",
    "categoria_produto",
    "data_compra",
    "quantidade",
    "valor",
]

window_spec = Window.partitionBy("cod_usuario", "categoria_produto").orderBy(
    "data_compra"
)

df_vendas_por_categoria = df_vendas_por_categoria.select(
    "cod_usuario",
    "cod_produto",
    "nome_produto",
    "categoria_produto",
    "data_compra",
    "quantidade",
    "valor",
    dense_rank().over(window_spec).alias("ordem_compra"),
)


df_vendas_por_categoria = df_vendas_por_categoria.withColumn(
    "valor_com_desconto",
    when(df_vendas_por_categoria.ordem_compra > 3, col("valor") * 0.9).otherwise(
        col("valor")
    ),
)


df_vendas_por_categoria = df_vendas_por_categoria.withColumn(
    "valor", format_number("valor", 2)
)

df_vendas_por_categoria = df_vendas_por_categoria.withColumn(
    "valor_com_desconto", format_number("valor_com_desconto", 2)
)


df_usuarios_a_exibir = df_vendas_por_categoria[["cod_usuario", "categoria_produto"]].filter(col("ordem_compra") > 3).dropDuplicates()

df_usuarios_a_exibir = df_usuarios_a_exibir.withColumnRenamed("cod_usuario", "cod_usuario_a_exibir")
df_usuarios_a_exibir = df_usuarios_a_exibir.withColumnRenamed("categoria_produto", "categoria_produto_a_exibir")


df_resposta = df_vendas_por_categoria.join(df_usuarios_a_exibir
                                           , (df_vendas_por_categoria.cod_usuario == df_usuarios_a_exibir.cod_usuario_a_exibir) &
                                           (df_vendas_por_categoria.categoria_produto == df_usuarios_a_exibir.categoria_produto_a_exibir) 
                                           , "inner")


df_resposta = df_resposta["cod_usuario"
                        , "produtos.cod_produto"
                        , "produtos.categoria_produto"
                        , "produtos.nome_produto"
                        , "data_compra"
                        , "ordem_compra"
                        , "valor"
                        , "valor_com_desconto"
                        ,]

df_resposta = df_resposta.withColumn("data_compra", date_format("data_compra", "yyyy-MM-dd"))

df_resposta = df_resposta.orderBy(
     asc("cod_usuario"), asc("categoria_produto"), desc("ordem_compra")
)


df_resposta.show( n= 50)

+-----------+-----------+--------------------+--------------------+-----------+------------+---------+------------------+
|cod_usuario|cod_produto|   categoria_produto|        nome_produto|data_compra|ordem_compra|    valor|valor_com_desconto|
+-----------+-----------+--------------------+--------------------+-----------+------------+---------+------------------+
|          3|          1|          Tecnologia|Notebook Asus Int...| 2021-07-20|           4|36,727.20|         33,054.48|
|          3|          3|          Tecnologia|  Smartphone Samsung| 2021-07-18|           3|20,495.00|         20,495.00|
|          3|          3|          Tecnologia|  Smartphone Samsung| 2021-04-14|           2|18,445.50|         18,445.50|
|          3|          1|          Tecnologia|Notebook Asus Int...| 2021-03-24|           1|18,363.60|         18,363.60|
|          6|          8|    Eletrodomesticos| Cafeteira Nespresso| 2021-03-23|           4|   439.14|            395.23|
|          6|          9

# FIM!